# VROOM testing without ORS
updated with vroom1.8.0 vroom-express 0.7, supporting Shipments

docs: https://github.com/VROOM-Project/vroom/blob/master/docs/API.md#shipments

In [104]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint

# DATI posizione consegne
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

vroom_url = 'http://vroom:3000/optimization/'

#centro mappa
center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI
point_start = [46.220, 11.106]

# POSIZIONE ARRIVO DEI VEICOLI(uguale a partenza)
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

#ELENCO consegne 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con quantita'
job_locs = ccA + ccB

#colori veicoli/percorsi by vid
Vcolors = {
    '1': 'lightblue',
    '2': 'red',
    '3': 'purple'
}

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

#convert coordinates (lat,lon) to (lon,lat)
def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def vehicle2Map(vid, cap, loc):
    mText = "vehicle "+str(vid)+"<br> capacity %s" % cap
    mColor = Vcolors[str(vid)]
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

vehicles = []

#veicolo 1
vid = 1
capacity = 6
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': revcc(point_start), # posizione di partenza veicoli
    'end': revcc(point_start)
})
vehicle2Map(vid, capacity, point_start)

#veicolo 2
vid = 2
capacity = 4
loc = [point_start[0], point_start[1] + 0.0040]
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': revcc(loc), # posizione di partenza veicoli
    'end': revcc(loc)
})
vehicle2Map(vid, capacity, point_startB)

#veicolo 3
vid = 3
capacity = 2
loc = [point_start[0], point_start[1] - 0.0045]
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': revcc(loc), # posizione di partenza veicoli
    'end': revcc(loc)
})
vehicle2Map(vid, capacity, loc)

jobs = []
for idx, cc in enumerate(job_locs):
    loc = revcc(cc)
    
    #quantita da consegnare
    delivery = random.randrange(1, 3)
    
    #tempo di della singola consegna(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    markerText = "id "+str(idx)+"<br> quantity %s <br> delivery time %s" % (delivery, toMin(service))
    mark = folium.Marker(loc, tooltip=folium.Tooltip(markerText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

    jobs.append({
        'id': idx,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        'service': service      # tempo di consegna
    })

Map

In [96]:
request = {
    'vehicles': vehicles,
    'jobs': jobs,
    'geometry': True
}

#request

In [97]:
resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
#solution

In [98]:
for route in solution['routes']:
    
    rtext = 'distance: '+toKm(route['distance'])+' delivery time: '+str(route['duration'])
    rcolor = Vcolors[str(route['vehicle'])]
    print('Vehicle'+str(route['vehicle']), rcolor, rtext)
        
    folium.PolyLine(
        locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
        color = rcolor
    ).add_to(Map)

print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Total time:', 'Not Delivered: '+str(len(solution['unassigned'])))

Map

Vehicle1 lightblue distance: 5km delivery time: 661
Vehicle2 red distance: 3km delivery time: 328

Total distance: 8km Total time: Not Delivered: 4


In [99]:
solution

{'code': 0,
 'summary': {'cost': 989,
  'unassigned': 4,
  'delivery': [9],
  'amount': [9],
  'pickup': [0],
  'service': 1266,
  'duration': 989,
  'waiting_time': 0,
  'priority': 0,
  'distance': 8172,
  'computing_times': {'loading': 18, 'solving': 9, 'routing': 22}},
 'unassigned': [{'id': 9, 'location': [11.114301681518556, 46.20900226913922]},
  {'id': 8, 'location': [11.122541427612305, 46.20986352455638]},
  {'id': 7, 'location': [11.127262115478516, 46.211972058400264]},
  {'id': 5, 'location': [11.118206977844238, 46.214911991592125]}],
 'routes': [{'vehicle': 1,
   'cost': 661,
   'delivery': [6],
   'amount': [6],
   'pickup': [0],
   'service': 826,
   'duration': 661,
   'waiting_time': 0,
   'priority': 0,
   'distance': 5018,
   'steps': [{'type': 'start',
     'location': [11.106, 46.22],
     'load': [6],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location': [11.092028617858888, 46.2173172744161],
     'id': 0,
     'servic